# 06 Clean Merged Data

In [14]:
import pandas as pd

In [2]:
df: pd.DataFrame = pd.read_csv('../data/interim/datasets_combined/all_teams_combined.csv')

In [3]:
df.head()

,Player,Season,Nationality,Pos,Position,Age,MP,Starts,Min,90s,...,Succ%,Tkld,Tkld%,Carries,CPA,Mis,Dis,Rec,MarketValueEuro,Current club
0,Abdellah Raihani,2324,Morocco,FW,Centre-Forward,19,1,0,10.0,0.1,...,NaN,0.0,NaN,1.0,0.0,1.0,0.0,1.0,300000.0,Atlético Madrid
1,Adrian Niño,2324,Spain,FW,Centre-Forward,19,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,500000.0,Atlético Madrid
2,Aitor Gismera,2324,Spain,MF,Central Midfield,19,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,250000.0,Atlético Madrid
3,Antoine Griezmann,2324,France,"FW,MF",Centre-Forward,32,33,31,2644.0,29.4,...,57.9,15.0,39.5,839.0,7.0,40.0,26.0,1194.0,25000000.0,Atlético Madrid
4,Antonio Gomis,2324,Spain,GK,Goalkeeper,20,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,150000.0,Atlético Madrid


---

## 1. Position data

Position data is messy:
- POS column has multiple values separated by commas
    * Could split into separate columns or create boolean flags
- Position column has "Centre-Forward" vs "Center Forward" spelling differences
    * Should group into Attack/Midfield/Defense/Goalkeeper categories

In [4]:
# Create Boolean Position Flags (for ML models)
def create_position_flags(df: pd.DataFrame) -> pd.DataFrame:
    """Create boolean flags for each position."""
    all_positions = ['FW', 'MF', 'DF', 'GK']
    for pos in all_positions:
        df[f'Pos_{pos}'] = df['Pos'].str.contains(pos, na=False).astype(int)
    return df

# Extract Primary Position (for simple analysis)
def get_primary_position(pos_string: str) -> str:
    """Extract the first position listed."""
    if pd.isna(pos_string):
        return 'Unknown'
    return pos_string.split(',')[0].strip()

In [5]:
# Apply all approaches
df = create_position_flags(df)
df['Primary_Pos'] = df['Pos'].apply(get_primary_position)

In [6]:
df.head()

,Player,Season,Nationality,Pos,Position,Age,MP,Starts,Min,90s,...,Mis,Dis,Rec,MarketValueEuro,Current club,Pos_FW,Pos_MF,Pos_DF,Pos_GK,Primary_Pos
0,Abdellah Raihani,2324,Morocco,FW,Centre-Forward,19,1,0,10.0,0.1,...,1.0,0.0,1.0,300000.0,Atlético Madrid,1,0,0,0,FW
1,Adrian Niño,2324,Spain,FW,Centre-Forward,19,0,0,NaN,NaN,...,NaN,NaN,NaN,500000.0,Atlético Madrid,1,0,0,0,FW
2,Aitor Gismera,2324,Spain,MF,Central Midfield,19,0,0,NaN,NaN,...,NaN,NaN,NaN,250000.0,Atlético Madrid,0,1,0,0,MF
3,Antoine Griezmann,2324,France,"FW,MF",Centre-Forward,32,33,31,2644.0,29.4,...,40.0,26.0,1194.0,25000000.0,Atlético Madrid,1,1,0,0,FW
4,Antonio Gomis,2324,Spain,GK,Goalkeeper,20,0,0,NaN,NaN,...,NaN,NaN,NaN,150000.0,Atlético Madrid,0,0,0,1,GK


Note: For ML models, consider dropping the original 'Pos' column since we now have:
- Boolean flags (Pos_FW, Pos_MF, Pos_DF, Pos_GK) - better for ML (represents all positions they can play with an arbitrary priority system)
- Primary_Pos - categorical but less granular
- Pos_Category - mapping by position (represents player's actual playing position)

The boolean flags are most suitable for ML as they avoid multicollinearity issues

In [7]:
# Create position category from Position column (more accurate)
position_category_mapping = {
    'Centre-Forward': 'Attack',
    'Second Striker': 'Attack',
    'Right Winger': 'Attack',
    'Left Winger': 'Attack',
    'Attacking Midfield': 'Attack',
    'Central Midfield': 'Midfield',
    'Defensive Midfield': 'Midfield',
    'Right Midfield': 'Midfield',
    'Left Midfield': 'Midfield',
    'Centre-Back': 'Defense',
    'Right-Back': 'Defense',
    'Left-Back': 'Defense',
    'Goalkeeper': 'Goalkeeper'
}
df['Pos_Category'] = df['Position'].map(position_category_mapping)
print(df['Pos_Category'].value_counts())

Pos_Category
Attack        214
Defense       206
Midfield      135
Goalkeeper     69
Name: count, dtype: int64


## 2. Nationality data

Nationality field needs work:
- FIFA codes are not ideal for ML (arbitrary, high cardinality)
- Better approach would be to use geographic coordinates (latitude, longitude)
- This captures meaningful distances between countries and cultures

In [15]:
# Geographic coordinates mapping for nationalities
nationality_coordinates = {
    'Spain': (40.0, -4.0),
    'Portugal': (39.0, -8.0),
    'France': (46.0, 2.0),
    'Belgium': (50.8, 4.0),
    'Netherlands': (52.1, 5.3),
    'Germany': (51.0, 10.0),
    'Austria': (47.3, 13.3),
    'Switzerland': (47.0, 8.0),
    'Italy': (42.8, 12.8),
    'Greece': (39.0, 22.0),
    'Croatia': (45.1, 15.2),
    'Slovenia': (46.0, 15.0),
    'Serbia': (44.0, 21.0),
    'Bosnia-Herzegovina': (44.0, 18.0),
    'North Macedonia': (41.6, 21.7),
    'Montenegro': (42.0, 19.0),
    'Romania': (46.0, 25.0),
    'Ukraine': (49.0, 32.0),
    'Poland': (52.0, 19.0),
    'Brazil': (-15.0, -47.0),
    'Argentina': (-34.6, -58.4),
    'Uruguay': (-34.9, -56.2),
    'Colombia': (4.7, -74.1),
    'Chile': (-33.4, -70.6),
    'Ecuador': (-0.2, -78.5),
    'Morocco': (34.0, -6.0),
    'Algeria': (36.8, 3.0),
    'Senegal': (14.7, -17.4),
    'Nigeria': (9.1, 7.4),
    'Ghana': (5.6, -0.2),
    'Guinea': (9.5, -13.7),
    'Gabon': (0.4, 9.5),
    'DR Congo': (-4.3, 15.3),
    'Cote d\'Ivoire': (5.3, -4.0),
    'Cape Verde': (14.9, -23.5),
    'Mozambique': (-25.9, 32.6),
    'United States': (38.9, -77.0),
    'Mexico': (19.4, -99.1),
    'Dominican Republic': (18.5, -69.9),
    'Guadeloupe': (16.3, -61.6),
    'England': (51.5, -0.1),
    'Denmark': (55.7, 12.6),
    'Norway': (59.9, 10.7),
    'Sweden': (59.3, 18.1),
    'Japan': (35.7, 139.7),
    'Georgia': (41.7, 44.8),
    'Türkiye': (39.9, 32.9),
    'Andorra': (42.5, 1.5)
}

In [16]:
# Add geographic coordinates to dataframe
df['Nationality_Latitude'] = df['Nationality'].map(lambda x: nationality_coordinates.get(x, (0, 0))[0])
df['Nationality_Longitude'] = df['Nationality'].map(lambda x: nationality_coordinates.get(x, (0, 0))[1])

print(f"Added latitude and longitude columns")
print(f"Sample coordinates:")
sample_coords = df[['Nationality', 'Nationality_Latitude', 'Nationality_Longitude']].drop_duplicates().head(10)
for _, row in sample_coords.iterrows():
    print(f"  {row['Nationality']}: ({row['Nationality_Latitude']:.1f}, {row['Nationality_Longitude']:.1f})")

Added latitude and longitude columns
Sample coordinates:
  Morocco: (34.0, -6.0)
  Spain: (40.0, -4.0)
  France: (46.0, 2.0)
  Belgium: (50.8, 4.0)
  Brazil: (-15.0, -47.0)
  Romania: (46.0, 25.0)
  Greece: (39.0, 22.0)
  Croatia: (45.1, 15.2)
  Slovenia: (46.0, 15.0)
  Uruguay: (-34.9, -56.2)


In [17]:
# Add distance features from major centers
major_centers = {
    'Europe': (48.0, 10.0),  # Central Europe
    'South_America': (-23.5, -46.6),  # São Paulo
    'Africa': (0.0, 20.0),  # Central Africa
    'North_America': (40.7, -74.0),  # New York
}

for center_name, (center_lat, center_lon) in major_centers.items():
    # Calculate rough distance (simplified)
    lat_diff = df['Nationality_Latitude'] - center_lat
    lon_diff = df['Nationality_Longitude'] - center_lon
    distance = (lat_diff**2 + lon_diff**2)**0.5
    df[f'Distance_from_{center_name}'] = distance

print(f"Added distance features from major footballing centers:")
distance_columns = [col for col in df.columns if col.startswith('Distance_from_')]
for col in distance_columns:
    print(f"  {col}")

Added distance features from major footballing centers:
  Distance_from_Europe
  Distance_from_South_America
  Distance_from_Africa
  Distance_from_North_America


In [18]:
# Show sample distances
print("Sample distances from major centers:")
sample_distances = df[['Nationality'] + distance_columns].drop_duplicates().head(5)
for _, row in sample_distances.iterrows():
    print(f"\n{row['Nationality']}:")
    for col in distance_columns:
        print(f"  {col}: {row[col]:.1f}")

# Remove FIFA codes column if it exists
if 'Nationality_FIFA' in df.columns:
    df = df.drop('Nationality_FIFA', axis=1)
    print("\nRemoved Nationality_FIFA column (not suitable for ML)")

print(f"\nFinal nationality features:")
nationality_features = [col for col in df.columns if 'Nationality' in col]
for feature in nationality_features:
    print(f"  {feature}")

print(f"\nTotal nationality features: {len(nationality_features)}")
print(f"Original cardinality: {df['Nationality'].nunique()} countries")
print(f"New cardinality: {len(nationality_features)} numeric features")

Sample distances from major centers:

Morocco:
  Distance_from_Europe: 21.3
  Distance_from_South_America: 70.4
  Distance_from_Africa: 42.8
  Distance_from_North_America: 68.3

Spain:
  Distance_from_Europe: 16.1
  Distance_from_South_America: 76.5
  Distance_from_Africa: 46.6
  Distance_from_North_America: 70.0

France:
  Distance_from_Europe: 8.2
  Distance_from_South_America: 84.8
  Distance_from_Africa: 49.4
  Distance_from_North_America: 76.2

Belgium:
  Distance_from_Europe: 6.6
  Distance_from_South_America: 89.9
  Distance_from_Africa: 53.3
  Distance_from_North_America: 78.7

Brazil:
  Distance_from_Europe: 85.0
  Distance_from_South_America: 8.5
  Distance_from_Africa: 68.7
  Distance_from_North_America: 61.9

Removed Nationality_FIFA column (not suitable for ML)

Final nationality features:
  Nationality
  Nationality_Latitude
  Nationality_Longitude

Total nationality features: 3
Original cardinality: 48 countries
New cardinality: 3 numeric features


In [19]:
df.head()

,Player,Season,Nationality,Pos,Position,Age,MP,Starts,Min,90s,...,Pos_DF,Pos_GK,Primary_Pos,Pos_Category,Nationality_Latitude,Nationality_Longitude,Distance_from_Europe,Distance_from_South_America,Distance_from_Africa,Distance_from_North_America
0,Abdellah Raihani,2324,Morocco,FW,Centre-Forward,19,1,0,10.0,0.1,...,0,0,FW,Attack,34.0,-6.0,21.260292,70.388991,42.801869,68.329276
1,Adrian Niño,2324,Spain,FW,Centre-Forward,19,0,0,NaN,NaN,...,0,0,FW,Attack,40.0,-4.0,16.124515,76.465744,46.647615,70.003500
2,Aitor Gismera,2324,Spain,MF,Central Midfield,19,0,0,NaN,NaN,...,0,0,MF,Midfield,40.0,-4.0,16.124515,76.465744,46.647615,70.003500
3,Antoine Griezmann,2324,France,"FW,MF",Centre-Forward,32,33,31,2644.0,29.4,...,0,0,FW,Attack,46.0,2.0,8.246211,84.806898,49.396356,76.184578
4,Antonio Gomis,2324,Spain,GK,Goalkeeper,20,0,0,NaN,NaN,...,0,1,GK,Goalkeeper,40.0,-4.0,16.124515,76.465744,46.647615,70.003500


NOTE: How to use these new features in ML models

**ML Feature Usage Notes**

**1. Position:**
* Boolean flags (Pos_FW, Pos_MF, Pos_DF, Pos_GK): Use all 4 for ML
* Primary_Pos: Drop (redundant with boolean flags)
* Pos_Category: Drop (redundant with boolean flags)
* Original 'Pos': Drop (high cardinality, messy)

**2. Nationality:**
* Nationality_Latitude, Nationality_Longitude: Use both
* Distance_from_*: Use all 4 distance features
* Original 'Nationality': Drop (high cardinality)

**3. Feature Selection for ML:**
* Keep: Pos_FW, Pos_MF, Pos_DF, Pos_GK (4 features)
* Keep: Nationality_Latitude, Nationality_Longitude (2 features)
* Keep: Distance_from_Europe, Distance_from_South_America, Distance_from_Africa, Distance_from_North_America (4 features)
* Total new features: 10 (vs 48+ original categorical features)

**4. Benefits:**
* Lower cardinality: 10 features vs 48+ categories
* Numeric features: Better for most ML algorithms
* Meaningful relationships: Geographic distances capture footballing culture similarities
* No multicollinearity: Boolean flags are independent

**5. DROP THESE COLUMNS BEFORE ML:**
* ['Pos', 'Primary_Pos', 'Pos_Category', 'Nationality']

## 3. Missing values

 Missing values everywhere:
- Need to figure out imputation strategy based on position/age
- Or just flag missing data for separate analysis
- Some players have no stats but high market values - need to investigate